In [5]:
import os
import shutil

import pandas as pd

import src.data.Dataset as dt

In [ ]:
FROM_PATH = os.path.abspath("./dataset/emptyNonEmptyDataset")
TO_PATH = os.path.abspath("./dataset/emptyNonEmptyDatasetFiltered")

ORIGINAL_DATASET = os.path.abspath("./data/raw/10000Images_binary.csv")
dataset_file_name = os.path.basename(ORIGINAL_DATASET)
FILTERED_PATH = os.path.abspath("./data/interim")
FILTERED_DATASET = os.path.join(FILTERED_PATH, dataset_file_name)

print(f"FROM_PATH:         {FROM_PATH}")
print(f"TO_PATH:           {TO_PATH}")
print(f"ORIGINAL_DATASET:  {ORIGINAL_DATASET}")
print(f"FILTERED_PATH:     {FILTERED_PATH}")
print(f"FILTERED_DATASET:  {FILTERED_DATASET}")

In [ ]:
dataset = dt.load_from_csv(ORIGINAL_DATASET)
dataset

In [ ]:
total_images_processed = 0
total_images_copied = 0

filtered_dataset = pd.DataFrame({}, columns=dataset.columns)

for index, row in dataset.iterrows():
    file_name = row["file_name"].replace("\\", "/")

    original_file = os.path.join(FROM_PATH, file_name[1:])
    
    filtered_file = file_name.lower()
    filtered_file = filtered_file.replace(" ","")
    filtered_file = filtered_file.replace("(", "_")
    filtered_file = filtered_file.replace(")", "_")
    filtered_file = filtered_file.replace("á","a").replace("é","e").replace("í","i").replace("ó","o").replace("ú","u")
    filtered_file = filtered_file.replace("ñ","n")
    
    new_row = pd.DataFrame({"file_name": [filtered_file[1:]], "label": [row["label"]]})
    filtered_dataset = pd.concat([filtered_dataset, new_row], ignore_index=True)
    
    filtered_file = os.path.join(TO_PATH, filtered_file[1:])

    os.makedirs(os.path.dirname(filtered_file), exist_ok=True)

    try:
        shutil.copyfile(original_file, filtered_file)
        total_images_copied += 1
    except FileNotFoundError:
        print(f"File not found: {original_file}")

    total_images_processed += 1

print(f"Total images processed: {total_images_processed}")
print(f"Total images copied: {total_images_copied}")

dt.dataset_to_csv(filtered_dataset, FILTERED_DATASET)